In [1]:
import formulas
import utilities
import topics
import pandas as pd
import os
import re


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    sort_orders_df = pd.read_csv(order_file_path)
    
    # Create formula-related markdown documents
    formulas_input_df = pd.read_csv(formula_file_path)
    formulas_df = formulas.get_formulas_df(formulas_input_df, sort_orders_df)
    formula_sheet_items = (
        formulas.get_formulas_on_formula_sheet(formulas_input_df))
    formula_proof_required_items = (
        formulas.get_formulas_where_proofs_required(formulas_input_df))    
    utilities.delete_directory_if_it_exists(docs_dir)
    formulas.create_formulas_content(
        formulas_df, formula_sheet_items, formula_proof_required_items, 
        sort_orders_df, docs_dir)
    formulas.create_calculus_summary(
        formulas_df, formula_sheet_items, formula_proof_required_items, 
        sort_orders_df, docs_dir)
    formulas.create_financial_summary(
        formulas_df, formula_sheet_items, formula_proof_required_items, 
        sort_orders_df, docs_dir)

    # Create topic related markdown documents
    topics_df = topics.get_topics_df(sort_orders_df)
    topics_dir = website_creator_dir + os.path.sep + 'topic_summaries'
    topics.create_topics_content(topics_df, sort_orders_df, docs_dir, 
                                 topics_dir)

""


""


""


""


""


KeyError: 'Level_1'

In [3]:
# Temp markdown file export of selected topic notebook

import utilities

input_file = '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/topic_summaries/financial_mathematics.ipynb'
output_file = '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/topic_summaries/temp.md'


export_string = utilities.filtered_notebook_md_string(
    input_notebook_path=input_file, include_tags=['Year 12 Adv'], 
    remove_input_tags=['hide_code'])

with open(output_file, "w") as text_file:
    text_file.write(export_string)

In [16]:
# example of kwargs pass through and modify

def fn_1(**kwargs):
    kwargs['c'] = 10
    fn_2(**kwargs)

def fn_2(a, b, c):
    print(a+b+c)

def fn_3(**kwargs):
    display(kwargs)
    
fn_1(a=1, b=2)
# fn_3(a=1,b=2,c=3)

13


In [10]:
def fn (a, **kwargs):
    print('test')

fn(b=1, c=3, a=2)


test
